In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Step 1: Load the dataset
df = pd.read_csv("students_data.csv")  # Make sure this CSV is in your working directory

# Step 2: Remove duplicates and missing values (if any)
df = df.drop_duplicates()
df = df.dropna()

# Step 3: Ensure proper data types
df['regno'] = df['regno'].astype(int)
df['cgpa'] = df['cgpa'].astype(float)

# Step 4: Encode categorical columns
le_dept = LabelEncoder()
le_arrear = LabelEncoder()

df['dept_encoded'] = le_dept.fit_transform(df['dept'])
df['hist_arrear_encoded'] = le_arrear.fit_transform(df['hist_arrear'])

# Show the encoding maps (optional)
print("Department Encoding Map:", dict(zip(le_dept.classes_, le_dept.transform(le_dept.classes_))))
print("Arrear Encoding Map:", dict(zip(le_arrear.classes_, le_arrear.transform(le_arrear.classes_))))

# Step 5: Feature Engineering
df['placement_eligible'] = (df['cgpa'] > 7.5) & (df['hist_arrear'] == 'No')
df['risk_group'] = df['cgpa'].apply(lambda x: 'High' if x < 6 else 'Low')

# Step 6: Save the cleaned and enriched dataset
df.to_csv("students_cleaned.csv", index=False)

print("✅ Preprocessing complete. File saved as students_cleaned.csv")

Department Encoding Map: {'AD': np.int64(0), 'CIVIL': np.int64(1), 'CSE': np.int64(2), 'ECE': np.int64(3), 'EEE': np.int64(4), 'FT': np.int64(5), 'MECH': np.int64(6)}
Arrear Encoding Map: {'No': np.int64(0), 'Yes': np.int64(1)}
✅ Preprocessing complete. File saved as students_cleaned.csv


In [4]:
import sqlite3
import pandas as pd

# Load cleaned data
df = pd.read_csv("students_cleaned.csv")

# Connect to SQLite database (or create it)
conn = sqlite3.connect("student_data.db")
cursor = conn.cursor()

# Create table
cursor.execute('''
CREATE TABLE IF NOT EXISTS students (
    regno INTEGER PRIMARY KEY,
    name TEXT,
    dept TEXT,
    cgpa REAL,
    hist_arrear TEXT,
    dept_encoded INTEGER,
    hist_arrear_encoded INTEGER,
    placement_eligible BOOLEAN,
    risk_group TEXT
)
''')

# Insert data into the table
df.to_sql('students', conn, if_exists='replace', index=False)

conn.commit()
conn.close()

print("✅ Database created and data inserted.")


✅ Database created and data inserted.


In [5]:
pip install flask

Note: you may need to restart the kernel to use updated packages.


In [7]:
from flask import Flask, render_template, request
import sqlite3
import pandas as pd

app = Flask(__name__)

def get_data():
    conn = sqlite3.connect("student_data.db")
    df = pd.read_sql("SELECT * FROM students", conn)
    conn.close()
    return df

@app.route("/")
def home():
    df = get_data()
    return render_template("index.html", tables=[df.to_html(classes='data')], titles=df.columns.values)

@app.route("/eligible")
def eligible():
    df = get_data()
    eligible_students = df[df['placement_eligible'] == True]
    return render_template("index.html", tables=[eligible_students.to_html(classes='data')], titles=eligible_students.columns.values)

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

In [8]:
<!DOCTYPE html>
<html>
<head>
    <title>Student Database</title>
    <style>
        .data { border-collapse: collapse; width: 100%; }
        .data th, .data td { border: 1px solid #ddd; padding: 8px; }
        .data th { background-color: #f2f2f2; }
    </style>
</head>
<body>
    <h2>📊 Student Data</h2>
    <a href="/">All Students</a> | <a href="/eligible">Placement Eligible Students</a>
    <br><br>
    {% for table in tables %}
        {{ titles[loop.index0] }}
        {{ table|safe }}
    {% endfor %}
</body>
</html>


SyntaxError: invalid decimal literal (3599481828.py, line 7)